## Pull in the data

In [4]:
# make sure to install these packages before running:
!pip install sodapy

In [3]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.texas.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.texas.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ubdr-4uff", limit=300000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
print(results_df.head())

  complaint_number                                    respondent_name  \
0                1                METROPOLITAN LIFE INSURANCE COMPANY   
1                2                       AETNA LIFE INSURANCE COMPANY   
2                3  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
3                4  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
4                5            CHARTER OAK FIRE INSURANCE COMPANY, THE   

  complainant_role                       reason complaint_confirmed_code  \
0         Relative  Cust Service Claim Handling                       No   
1         Provider     Delays (Claims Handling)                       No   
2         Provider              Denial Of Claim                       No   
3         Provider              Denial Of Claim                       No   
4          Insured  Unsatisfactory Settle/Offer                       No   

                                       disposition            received_date  \
0                        

In [5]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258228 entries, 0 to 258227
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   complaint_number          258228 non-null  object
 1   respondent_name           258228 non-null  object
 2   complainant_role          258228 non-null  object
 3   reason                    258222 non-null  object
 4   complaint_confirmed_code  258223 non-null  object
 5   disposition               257191 non-null  object
 6   received_date             258228 non-null  object
 7   closed_date               258228 non-null  object
 8   complaint_type            258227 non-null  object
 9   coverage_type             258228 non-null  object
 10  coverage_level            258228 non-null  object
 11  respondent_id             258228 non-null  object
 12  respondent_role           258226 non-null  object
 13  respondent_type           258228 non-null  object
 14  comp

In [12]:
df = pd.read_csv('Insurance_complaints__All_data_20241020.csv')

In [13]:
df.info()

   Complaint number                            Complaint filed against  \
0                 1                METROPOLITAN LIFE INSURANCE COMPANY   
1                 2                       AETNA LIFE INSURANCE COMPANY   
2                 3  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
3                 4  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
4                 5            CHARTER OAK FIRE INSURANCE COMPANY, THE   

  Complaint filed by       Reason complaint filed Confirmed complaint  \
0           Relative  Cust Service Claim Handling                  No   
1           Provider     Delays (Claims Handling)                  No   
2           Provider              Denial Of Claim                  No   
3           Provider              Denial Of Claim                  No   
4            Insured  Unsatisfactory Settle/Offer                  No   

                                      How resolved Received date Closed date  \
0                                   

In [15]:
df['How resolved'].value_counts()

How resolved
Contract Language/Legal Issue                                                 25697
No Jurisdiction                                                               23532
Information Furnished                                                         17884
Question of Fact                                                              12207
Information Furnished; Contract Language/Legal Issue                           8035
                                                                              ...  
Policy Issued/Restored; Corrective Action Taken; Cancellation Withdrawn           1
Agent Resp Recvd; Contract Language/Legal Issue; Failure to Timely Respond        1
Corrective Action Taken; Company Position Upheld; Claim Settled                   1
Information Furnished; Premium Refunded; Failure to Timely Respond                1
Agent Resp Recvd; Agent 2nd warning; Information Furnished                        1
Name: count, Length: 4064, dtype: int64

In [20]:
x = df.drop(columns=['How resolved'])
print(x.head())

   Complaint number                            Complaint filed against  \
0                 1                METROPOLITAN LIFE INSURANCE COMPANY   
1                 2                       AETNA LIFE INSURANCE COMPANY   
2                 3  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
3                 4  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
4                 5            CHARTER OAK FIRE INSURANCE COMPANY, THE   

  Complaint filed by       Reason complaint filed Confirmed complaint  \
0           Relative  Cust Service Claim Handling                  No   
1           Provider     Delays (Claims Handling)                  No   
2           Provider              Denial Of Claim                  No   
3           Provider              Denial Of Claim                  No   
4            Insured  Unsatisfactory Settle/Offer                  No   

  Received date Closed date             Complaint type        Coverage type  \
0    06/12/2012  07/25/2012  Life, Ac

In [23]:
y = df['How resolved']
print(y.head())

0                                              Other
1                              Information Furnished
2                                              Other
3                                              Other
4    Question of Fact; Contract Language/Legal Issue
Name: How resolved, dtype: object


In [24]:
# Handle missing values
df.fillna({'Keywords': 'Unknown', 'Others involved': 'None'}, inplace=True)
df.dropna(subset=['Complaint filed against', 'Reason complaint filed', 'How resolved'], inplace=True)

# Convert data types
df['Received date'] = pd.to_datetime(df['Received date'])
df['Closed date'] = pd.to_datetime(df['Closed date'])

# Normalize text data
df['Complaint filed against'] = df['Complaint filed against'].str.upper()
df['Reason complaint filed'] = df['Reason complaint filed'].str.lower()
df['How resolved'] = df['How resolved'].str.lower()

# Feature engineering: Create a new feature for the duration of the complaint
df['Complaint duration'] = (df['Closed date'] - df['Received date']).dt.days

# Remove duplicates
df.drop_duplicates(inplace=True)

# Encode categorical variables
df = pd.get_dummies(df, columns=['Complaint type', 'Coverage type', 'Coverage level', 'Complainant type'])

print(df.head())

   Complaint number                            Complaint filed against  \
0                 1                METROPOLITAN LIFE INSURANCE COMPANY   
1                 2                       AETNA LIFE INSURANCE COMPANY   
2                 3  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
3                 4  BLUE CROSS AND BLUE SHIELD OF TEXAS, A DIVISIO...   
4                 5            CHARTER OAK FIRE INSURANCE COMPANY, THE   

  Complaint filed by       Reason complaint filed Confirmed complaint  \
0           Relative  cust service claim handling                  No   
1           Provider     delays (claims handling)                  No   
2           Provider              denial of claim                  No   
3           Provider              denial of claim                  No   
4            Insured  unsatisfactory settle/offer                  No   

                                      How resolved Received date Closed date  \
0                                   

In [26]:
print(df.describe())

       Complaint number                  Received date  \
count     257186.000000                         257186   
mean      200623.872765  2018-03-22 00:28:11.812151808   
min            1.000000            2011-04-28 00:00:00   
25%        98874.250000            2015-06-09 00:00:00   
50%       198667.500000            2017-12-12 00:00:00   
75%       306854.750000            2021-02-10 00:00:00   
max       407369.000000            2024-10-18 00:00:00   
std       117185.542540                            NaN   

                         Closed date  Respondent ID  Complaint duration  
count                         257186   2.571860e+05       257186.000000  
mean   2018-07-03 04:43:48.646971392   4.913579e+04          103.177510  
min              2012-05-21 00:00:00   5.200000e+01            0.000000  
25%              2015-09-01 00:00:00   1.348000e+03           35.000000  
50%              2018-07-31 00:00:00   2.737000e+03           70.000000  
75%              2021-03-10 00:00

# Loading the Data

In [2]:
# import required libraries
import pandas as pd
from sodapy import Socrata
import numpy as np

In [3]:
df = pd.read_csv('Insurance_complaints__All_data_20241020.csv')

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258228 entries, 0 to 258227
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Complaint number         258228 non-null  int64 
 1   Complaint filed against  258224 non-null  object
 2   Complaint filed by       258228 non-null  object
 3   Reason complaint filed   258222 non-null  object
 4   Confirmed complaint      258223 non-null  object
 5   How resolved             257191 non-null  object
 6   Received date            258228 non-null  object
 7   Closed date              258228 non-null  object
 8   Complaint type           258227 non-null  object
 9   Coverage type            258228 non-null  object
 10  Coverage level           258228 non-null  object
 11  Others involved          229881 non-null  object
 12  Respondent ID            258228 non-null  int64 
 13  Respondent Role          258226 non-null  object
 14  Respondent type     

In [5]:
# drop columns with more than 50% missing values
df = df.dropna(thresh=0.5*len(df), axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258228 entries, 0 to 258227
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Complaint number         258228 non-null  int64 
 1   Complaint filed against  258224 non-null  object
 2   Complaint filed by       258228 non-null  object
 3   Reason complaint filed   258222 non-null  object
 4   Confirmed complaint      258223 non-null  object
 5   How resolved             257191 non-null  object
 6   Received date            258228 non-null  object
 7   Closed date              258228 non-null  object
 8   Complaint type           258227 non-null  object
 9   Coverage type            258228 non-null  object
 10  Coverage level           258228 non-null  object
 11  Others involved          229881 non-null  object
 12  Respondent ID            258228 non-null  int64 
 13  Respondent Role          258226 non-null  object
 14  Respondent type     

In [7]:


import pandas as pd
from sodapy import Socrata

client = Socrata("data.texas.gov", None)

results = client.get("ubdr-4uff", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [9]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   complaint_number          2000 non-null   object
 1   respondent_name           2000 non-null   object
 2   complainant_role          2000 non-null   object
 3   reason                    2000 non-null   object
 4   complaint_confirmed_code  2000 non-null   object
 5   disposition               1948 non-null   object
 6   received_date             2000 non-null   object
 7   closed_date               2000 non-null   object
 8   complaint_type            2000 non-null   object
 9   coverage_type             2000 non-null   object
 10  coverage_level            2000 non-null   object
 11  respondent_id             2000 non-null   object
 12  respondent_role           2000 non-null   object
 13  respondent_type           2000 non-null   object
 14  complainant_type        

In [10]:
# Drop missing rows in disposition column
results_df.dropna(subset=['disposition'], inplace=True)

In [13]:
print(results_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1948 entries, 0 to 1999
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   complaint_number          1948 non-null   object
 1   respondent_name           1948 non-null   object
 2   complainant_role          1948 non-null   object
 3   reason                    1948 non-null   object
 4   complaint_confirmed_code  1948 non-null   object
 5   disposition               1948 non-null   object
 6   received_date             1948 non-null   object
 7   closed_date               1948 non-null   object
 8   complaint_type            1948 non-null   object
 9   coverage_type             1948 non-null   object
 10  coverage_level            1948 non-null   object
 11  respondent_id             1948 non-null   object
 12  respondent_role           1948 non-null   object
 13  respondent_type           1948 non-null   object
 14  complainant_type          194

In [ ]:
# drop Unnecessary columns
results_df.drop(columns=['complaint_number', 'respondent_id', 

Index(['complaint_number', 'respondent_name', 'complainant_role', 'reason',
       'complaint_confirmed_code', 'disposition', 'received_date',
       'closed_date', 'complaint_type', 'coverage_type', 'coverage_level',
       'respondent_id', 'respondent_role', 'respondent_type',
       'complainant_type', 'involved_party_type', 'keyword'],
      dtype='object')